In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.graph.message import AnyMessage
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_core.messages import HumanMessage
from langgraph.graph.message import add_messages, MessagesState
from typing import Annotated
from langchain_google_genai import ChatGoogleGenerativeAI


/Users/abdulvajid/AI/PROJECTS/Simple-Multi-turn-ChatBot-with-langgraph/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# os.environ['GOOGLE_API_KEY']

In [4]:
class State(MessagesState):
    pass
    

In [5]:
graph = StateGraph(State)

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
)

In [7]:
def model_call(state):
    return {"messages" : [llm.invoke(state['messages'])]}

In [10]:
graph.set_entry_point('node1')
graph.add_node('node1', model_call)
graph.add_edge('node1', END)

In [11]:
graph = graph.compile()

In [14]:
type(graph)

langgraph.graph.state.CompiledStateGraph

In [15]:
messages = [HumanMessage('can you explain me how you get build')]

In [17]:
state = graph.invoke({'messages': messages})

In [22]:
new_message = [HumanMessage('ok now explain me how can i do it')]

In [24]:
state['messages'] + new_message

[HumanMessage(content='can you explain me how you get build', additional_kwargs={}, response_metadata={}, id='28efb95c-a2f1-4bb9-9d12-ba979ee3a105'),
 AIMessage(content='Okay, let\'s break down how I, as a large language model, am "built." It\'s a multi-faceted process involving data, architecture, training, and ongoing refinement.  Think of it like building a house: you need materials (data), a blueprint (architecture), construction workers (training algorithms), and ongoing maintenance (refinement).\n\n**1. Data: The Foundation**\n\n*   **Massive Text and Code Datasets:** I am trained on an enormous amount of text and code data. This data comes from a variety of sources, including:\n    *   **The Public Web:**  Web pages, articles, blog posts, forums, and other publicly available text.\n    *   **Books:**  A vast collection of books covering a wide range of subjects.\n    *   **Code Repositories:**  Code from platforms like GitHub, covering various programming languages.\n    *   **O

In [26]:
output = graph.invoke({'messages': state['messages'] + new_message})

In [31]:
print(output['messages'][-1].content)

Okay, let's break down how you can build your own language model. Keep in mind that building a state-of-the-art model like me requires significant resources (data, compute, expertise). However, you can definitely build smaller, simpler models to learn the fundamentals and experiment. Here's a roadmap:

**1. Define Your Goal and Scope:**

*   **What do you want your model to do?**  Be specific.  Examples:
    *   **Text generation:** Generate creative text, stories, poems, code, etc.
    *   **Text classification:** Classify text into categories (e.g., spam detection, sentiment analysis).
    *   **Question answering:** Answer questions based on a given context.
    *   **Translation:** Translate text from one language to another.
    *   **Summarization:** Summarize long documents.
*   **What is your target domain?**  Will it be general-purpose, or focused on a specific area (e.g., medical text, legal documents, financial news)?
*   **What resources do you have available?**  Consider y